<img width="10%" alt="Naas" src="https://landen.imgix.net/jtci2pxwjczr/assets/5ice39g4.png?w=160"/>


# LinkedIn - List top company content fans
<a href="https://app.naas.ai/user-redirect/naas/downloader?url=https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/LinkedIn/Linkedin_Follow_number_of_content_published.ipynb" target="_parent"><img src="https://naasai-public.s3.eu-west-3.amazonaws.com/open_in_naas.svg"/></a>

**Tags:** #linkedin #html #plotly #csv #image #content #analytics #automation

**Author:** [Florent Ravenel](https://www.linkedin.com/in/florent-ravenel/)

This notebook follows the evolution of content shares on LinkedIn.

## Input

In [1]:
# Except allow to run common.ipynb once using Naas Engine
try:
    %run "../common.ipynb"
except:
    %run "common.ipynb"

### Setup Variables

In [2]:
KPI_TITLE = "Top fans"
INPUT_DATABASE = LK_COMPANY_ENGAGEMENTS
OWNER = LK_COMPANY_NAME
OWNER_ID = LK_COMPANY_ID
PLATFORM = LINKEDIN
LOGO = LINKEDIN_LOGO
COLOR = LINKEDIN_COLOR

### Setup Constants

In [3]:
# Inputs
PERIOD = PERIOD_MTD

## Model

### Get your engagements

In [4]:
df_engagements = get_data(INPUT_DATABASE)
print("✅ Engagements fetched:", len(df_engagements))
df_engagements.head(1)

✅ Engagements fetched: 70


,PROFILE_ID,PROFILE_URL,PUBLIC_ID,FIRSTNAME,LASTNAME,FULLNAME,OCCUPATION,REACTION,TEXT,TITLE,PUBLISHED_DATE,AUTHOR_NAME,POST_URL
0,ACoAABYVsxEBFJ5cAeOCukhSw2dAK8h_pMtwg1E,https://www.linkedin.com/in/ACoAABYVsxEBFJ5cAe...,valentin-piquard-955105a4,Valentin,Piquard,Valentin Piquard,Product-Led Growth @Naas.ai | Building an Open...,LIKES,Not defined,No comment. 👌,2022-06-26 16:22:00+02:00,naas.ai,https://www.linkedin.com/feed/update/urn:li:ac...


### Get fans

In [5]:
def get_fans(df):
    # Init
    df_fans = df.copy()
    
    # Filter
    df_fans["SCENARIO"] = pd.to_datetime(df_fans["PUBLISHED_DATE"].str[:-6]).dt.strftime(PERIOD)
    df_fans = df_fans[(df_fans["SCENARIO"] == datetime.now().strftime(PERIOD)) & (df_fans["FULLNAME"] != OWNER)].reset_index(drop=True)

    # Groupby
    to_group = ["PROFILE_URL", "FULLNAME", "OCCUPATION"]
    to_agg = {"PROFILE_ID": "count"}
    
    df_fans = df_fans.groupby(to_group, as_index=False).agg(to_agg)
    df_fans = df_fans.sort_values(by="PROFILE_ID", ascending=False).reset_index(drop=True)[:20]
    return df_fans

df_fans = get_fans(df_engagements)
df_fans

,PROFILE_URL,FULLNAME,OCCUPATION,REACTION,PROFILE_ID
0,https://www.linkedin.com/in/ACoAABYVsxEBFJ5cAe...,Valentin Piquard,Product-Led Growth @Naas.ai | Building an Open...,LIKES,7
1,https://www.linkedin.com/in/ACoAADECqNcBio1wz7...,Shruthi Jain,|Data Analyst at Xeno|,LIKES,4
2,https://www.linkedin.com/in/ACoAABCNSioBW3YZHc...,Florent Ravenel,COO @Naas.ai | Automation and data templates i...,LIKES,3
3,https://www.linkedin.com/in/ACoAAABFvlkBIKRZTA...,Debbie Botha,Women in AI Global Chief Partnership Officer,LIKES,2
4,https://www.linkedin.com/in/ACoAAAJ-AnwBgyHAkS...,Raouf Mazouz,Data Scientist | ERP Expert | ML & AI Enthusia...,LIKES,2
5,https://www.linkedin.com/in/ACoAAA6EYJABlJdZG2...,Maxime Jublou,"CTO @naas.ai | Cloud, DevOps and Software deve...",LIKES,2
6,https://www.linkedin.com/in/ACoAAAJHE7sB5OxuKH...,Jérémy Ravenel,⚡️ Building @naas.ai - open source data platfo...,LIKES,2
7,https://www.linkedin.com/in/ACoAAByNUAYBPZEpdf...,Waleed Elhardallou,Digital Marketing Specialist at ShareColours.Se,LIKES,2
8,https://www.linkedin.com/in/ACoAAC0ndQIBTW3BMy...,Peace Bello,Chemical Engineering Finalist|| Process Optimi...,LIKES,2
9,https://www.linkedin.com/in/ACoAAAmO4lgBYd3oqz...,Shaheeb Rahman,Data Engineering Architect with 12 yrs experie...,LIKES,2


## Output

### Update Notion Report

In [6]:
def update_report_status(report,
                         platform,
                         owner,
                         df=pd.DataFrame(),
                         image_link=None,
                         html_link=None,
                         csv_link=None,
                         page_icon=LOGO,
                         ):
    # Decode database id
    database_id = naas.secret.get(name="NOTION_DATABASE_URL").split("/")[-1].split("?v=")[0]
    
    # Get pages from notion database
    pages = notion.connect(NOTION_TOKEN).database.query(database_id)
    
    # Create or update page
    page_new = True
    for page in pages:
        page_temp = page.df()
        page_id = page_temp.loc[page_temp.Name == "Report name", "Value"].values
        page_id2 = page_temp.loc[page_temp.Name == "Platform", "Value"].values
        page_id3 = page_temp.loc[page_temp.Name == "Owner", "Value"].values
        if page_id == report and page_id2 == platform and page_id3 == owner:
            page_new = False
            break
    try:
        if page_new:
            page = notion.connect(NOTION_TOKEN).Page.new(database_id=database_id).create()
            page.title("Report name", report)
            page.multi_select("Platform", [platform])
            page.select("Owner", owner)
            notion.client.pages.update(page_id=page.id, icon={'type': 'external', 'external': {'url': page_icon}})
            
        # Check if image already exists
        blocks = page.get_blocks()
        if len(blocks) > 0:
            while True:
                for block in blocks:
                    notion.connect(NOTION_TOKEN).blocks.delete(block.id)
                blocks = page.get_blocks()
                if len(blocks) == 0:
                    break
        page.update()
        
        
        for index, row in df.iterrows():
            text = f"{row.FULLNAME} - {row.OCCUPATION} ({str(row.PROFILE_ID)})"
            res = page.numbered_list_item(text)
            res.numbered_list_item.text[0].href = row.PROFILE_URL
            res.numbered_list_item.text[0].text.link = Link(row.PROFILE_URL)

        # Create page in Notion
        page.date("Last updated date", datetime.now(pytz.timezone(TIMEZONE)).strftime("%Y-%m-%d %H:%M:%S%z"))
        page.update()
        print(f"✅ Page '{report} - {platform} - {owner}' updated in Notion.")
    except Exception as e:
        raise(e)

update_report_status(KPI_TITLE,
                     PLATFORM,
                     OWNER,
                     df_fans)

✅ Page 'Top fans - LinkedIn - naas.ai' updated in Notion.
